In [ ]:
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [ ]:
sentence = "unsure"
input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
input_ids

tensor([[13271,   495]])

In [ ]:
tokenizer.decode(input_ids[0])

'unsure'

In [ ]:
sentence = "unbelivable"
input_ids = tokenizer(sentence, return_tensors="pt").input_ids
input_ids

tensor([[  403,  6667, 21911]])

In [ ]:
for token_id in input_ids[0]:
    print(tokenizer.decode(token_id))

un
bel
ivable


In [ ]:
word = "homoscedasticity"
my_ids = tokenizer(word, return_tensors="pt").input_ids
my_ids

tensor([[26452,   418,   771,  3477,   414]])

In [ ]:
tokenizer.decode(my_ids.squeeze())

'homoscedasticity'

In [ ]:
word = "pneumonoultramicroscopicsilicovolcanoconiosis"
my_ids = tokenizer(word, return_tensors="pt").input_ids
# len(my_ids[0])
my_ids

tensor([[   79, 25668,   261, 25955,   859,  2500,  1416,   404,   873, 41896,
           709,   349,  5171, 36221, 42960]])

In [ ]:
for token_id in my_ids.squeeze():
    print(tokenizer.decode(token_id))

p
neum
on
oult
ram
icro
sc
op
ics
ilic
ov
ol
can
ocon
iosis


In [ ]:
sentence = "antidisestablishmentarianism"
token_ids = tokenizer(sentence, return_tensors="pt").input_ids
token_ids, len(token_ids[0])

(tensor([[  415, 29207, 44390,  3699,  1042]]), 5)

In [ ]:
word = "floccinaucinihilipilification"

my_ids = tokenizer(word, return_tensors="pt").input_ids
my_ids, len(my_ids[0])

(tensor([[ 2704, 13966,   259, 14272,   259, 20898,   541,   346,  2649]]), 9)

In [ ]:
for token_id in my_ids[0]:
    print(tokenizer.decode(token_id))

fl
occ
in
auc
in
ihil
ip
il
ification


In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
gpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
sentence = "I like machine learning to be able to predict the future."
# Data Processing technique
token_ids = tokenizer(sentence, return_tensors="pt").input_ids

outputs = gpt2(token_ids).logits[0, -1]
tokenizer.decode(outputs.argmax())

'\n'

In [ ]:
sentence = "I learn machine learning to enhance our understanding of the world around us."

token_ids = tokenizer(sentence, return_tensors="pt").input_ids

outputs = gpt2(token_ids).logits[0, -1]
tokenizer.decode(outputs.argmax())

'\n'

In [ ]:
sentence = "I learn machine learning to enhance"
token_ids = tokenizer(sentence, return_tensors="pt").input_ids
outputs = gpt2(token_ids).logits[0, -1]
final_logits = torch.topk(outputs, 20) # Feel free to play around with the K

for index in final_logits.indices:
    print(tokenizer.decode(index))

 our
 the
 my
 your
 human
 a
 and
 its
 their
 performance
 learning
 user
 data
 social
 real
 understanding
 information
,
 intelligence
 people


In [ ]:
torch.softmax(final_logits.values, dim=0).sum()

tensor(1.0000, grad_fn=<SumBackward0>)

In [ ]:
torch.softmax(final_logits.values, dim=0).sum()

tensor(1.0000, grad_fn=<SumBackward0>)

In [ ]:
def greedy_decode(logits):
    """Return token index with maximum probability."""
    return torch.argmax(logits, dim=-1)

# TOP K SAMPLING

def top_k_sampling(logits, k=50):
    """
    keeps only top-k logits, normalize them into probability.
    them sample one token from the filtered distribution.
    """
    values, indices = torch.topk(logits, k)
    probs = F.softmax(values, dim=-1)
    sampled = torch.multinomial(probs, 1)
    return indices[sampled]

# Top-p (Nuecles) Sampling

def top_p_sampling(logits, p=0.9):
    """
    Sort tokens by probability, keep smallest number whose culumative
    probability exceeds threshold p, then sample one token.
    """

    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    sorted_probs = F.softmax(sorted_logits, dim=-1)
    cumulative_probs = sorted_probs.cumsum(dim=-1)

    # Mask token outside nuclues
    mask = cumulative_probs > p
    sorted_logits[mask] = float("-inf")

    # Sample from filtered logits
    filtered_probs = F.softmax(sorted_logits, dim=-1)
    sampled = torch.multinomial(filtered_probs, 1)

    # Return token index in originial vocabulary
    return sorted_indices[sampled]

## Temperature Sampling ##

def temperature_sampling(logits, temperature=1.0):
    """
    Scale logits by temperature before sampling.
    Lower temperature => sharper distribution
    """

    scaled = logits / temperature
    probs = F.softmax(scaled, dim=-1)
    return torch.multinomial(probs, 1)


## Random Sampling ##

def random_sampling(logits):
    """
    Sample dirctly from softmax distribution without filtring
    """

    probs = F.softmax(logits, dim=-1)
    return torch.multinomial(probs, 1)

# sentence = "Today I decided to go to the local library and find out what was in my wallet."
sentence = "I am really happy becuase I have gone back in time."
inputs = tokenizer(sentence, return_tensors="pt")
output = gpt2(**inputs)
logits = output.logits[0, -1]

print(f"Greedy Decode: ", tokenizer.decode([greedy_decode(logits)]))
print(f"Top-K Sampling: ", tokenizer.decode(top_k_sampling(logits, k=10)))
print(f"Top-P-Sampling: ", tokenizer.decode(top_p_sampling(logits, p=0.9)))
print(f"Temp: ", tokenizer.decode(temperature_sampling(logits, temperature=1)))
print(f"Radnom: ", tokenizer.decode(random_sampling(logits)))

Greedy Decode:   time
Top-K Sampling:   time
Top-P-Sampling:   time
Temp:   time
Radnom:   time


In [ ]:
tokenizer.decode(top_k_sampling(outputs))

' my'

In [ ]:
outputs

tensor([-109.4038, -108.0284, -114.4298,  ..., -111.9259, -117.0069,
        -109.9712], grad_fn=<SelectBackward0>)

In [ ]:
tokenizer.decode(top_p_sampling(outputs, p=0.9))

' my'

In [ ]:
tokenizer.decode(temperature_sampling(outputs, temperature=1.5))

' detection'

In [ ]:
tokenizer.decode(random_sampling(outputs))

' complaints'

In [ ]:
sentence = "I learn machine learning to enhance our understanding of the brain in"
token_ids = tokenizer(sentence, return_tensors="pt").input_ids
outputs = gpt2(token_ids).logits # Raw Unnormlized Score - Values
outputs = torch.softmax(outputs[0, -1], dim=-1)

top10 = torch.topk(outputs, k=10)

for index, value in zip(top10.indices, top10.values):
    print(f"{tokenizer.decode(index)} -- {value:.1%}")

 a -- 14.7%
 the -- 7.7%
 order -- 6.2%
 ways -- 6.2%
 general -- 4.5%
 humans -- 2.9%
 many -- 2.8%
 an -- 2.4%
 our -- 2.3%
 everyday -- 1.7%


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
prompt = "What is machine learning?"
output = pipe(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(output[0]["generated_text"])

What is machine learning? It's not very hard to understand. It's one of the reasons why we've been able to develop the neural network for some time. But it's not very difficult, and it's not that hard to understand. We've been able to get a lot of work done here. It's only been a couple of weeks on the project, but it's been really exciting.

Why are we using machine learning?

A lot of the work we've done so far has been focused on machine learning, where we were able to build out the neural network in a way that was better than what we were able to build out on our own. Because the neural network is basically a machine, as opposed to a human. The idea is to get a better understanding of how the brain works.

There is a lot of work to be done on machine learning. There is a lot of work I'm doing on machine learning with the Watson. I don't know what Watson is, but there are some things we can do to get around this problem. For example, there is a set of questions that we're trying to 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Sentiment Analysis

In [ ]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
type(ds)

datasets.dataset_dict.DatasetDict

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
ds["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
import pandas as pd

In [ ]:
ds["train"].to_pandas()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [ ]:
my_dataset_df = ds["train"].to_pandas()

In [ ]:
my_dataset_df["text"]

,text
0,I rented I AM CURIOUS-YELLOW from my video sto...
1,"""I Am Curious: Yellow"" is a risible and preten..."
2,If only to avoid making this type of film in t...
3,This film was probably inspired by Godard's Ma...
4,"Oh, brother...after hearing about this ridicul..."
...,...
24995,A hit at the time but now better categorised a...
24996,I love this movie like no other. Another time ...
24997,This film and it's sequel Barry Mckenzie holds...
24998,'The Adventures Of Barry McKenzie' started lif...


In [ ]:
len(my_dataset_df["text"])

25000

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
classifier("This day is great!")

[{'label': 'POSITIVE', 'score': 0.999881386756897}]

In [ ]:
classifier("This day is terrible and i am so sad")[0]["label"]

'NEGATIVE'

In [ ]:
def score(review_text):
    return classifier(review_text[:500])[0]["label"]

In [ ]:
my_dataset_df["model_prediction"] = my_dataset_df["text"].apply(score)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
my_dataset_df[["label", "model_prediction"]][:20]

,label,model_prediction
0,0,NEGATIVE
1,0,NEGATIVE
2,0,NEGATIVE
3,0,POSITIVE
4,0,NEGATIVE
5,0,NEGATIVE
6,0,NEGATIVE
7,0,NEGATIVE
8,0,NEGATIVE
9,0,NEGATIVE


In [ ]:
my_dataset_df.iloc[0]

,0
text,I rented I AM CURIOUS-YELLOW from my video sto...
label,0
model_prediction,NEGATIVE


In [ ]:
review = my_dataset_df.iloc[0]["text"]
classifier(review)[0]["label"]

'POSITIVE'

In [ ]:
from transformers import pipeline

In [ ]:
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
sentence = "The company reported a strong increase in quarterly revnue, exceeding market expectations."
finbert(sentence)

[{'label': 'positive', 'score': 0.951565146446228}]

In [ ]:
sentence = "Shares fell after the firm reported lower-than-expected earnings"
finbert(sentence)

[{'label': 'negative', 'score': 0.9718543887138367}]

In [ ]:
sentences = ["Strong consumer demand drove record sales across all regions",
             "Supply chain disruptions severly affected production output"]

In [ ]:
finbert(sentences)

[{'label': 'positive', 'score': 0.9434744715690613},
 {'label': 'negative', 'score': 0.9608097672462463}]

### Named Entity Recognition

In [ ]:
sentence = "Apple announced record earnings in the United States on Monday."

In [ ]:
ner = pipeline("ner")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
sentence

'Apple announced record earnings in the United States on Monday.'

In [ ]:
ner(sentence)

[{'entity': 'I-ORG',
  'score': np.float32(0.9991387),
  'index': 1,
  'word': 'Apple',
  'start': 0,
  'end': 5},
 {'entity': 'I-LOC',
  'score': np.float32(0.9997105),
  'index': 7,
  'word': 'United',
  'start': 39,
  'end': 45},
 {'entity': 'I-LOC',
  'score': np.float32(0.99967074),
  'index': 8,
  'word': 'States',
  'start': 46,
  'end': 52}]

In [ ]:
sentence = "I live in UK worked at Facebook after graduating from Harvard"

In [ ]:
ner(sentence)

[{'entity': 'I-LOC',
  'score': np.float32(0.9997962),
  'index': 4,
  'word': 'UK',
  'start': 10,
  'end': 12},
 {'entity': 'I-ORG',
  'score': np.float32(0.9983279),
  'index': 7,
  'word': 'Facebook',
  'start': 23,
  'end': 31},
 {'entity': 'I-ORG',
  'score': np.float32(0.9209187),
  'index': 11,
  'word': 'Harvard',
  'start': 54,
  'end': 61}]

## Question Answering

In [ ]:
qa_bot = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
context = """
Financial sentiment analysis is a challenging task due to the specialized
language and lack of labeled data in that domain. General-purpose models are
not effective enough because of the specialized language used in a financial
context. We hypothesize that pre-trained language models can help with this
problem because they require fewer labeled examples and they can be further
trained on domain-specific corpora. We introduce FinBERT, a language model
based on BERT, to tackle NLP tasks in the financial domain. Our results show
improvement in every measured metric on current state-of-the-art results for
two financial sentiment analysis datasets. We find that even with a smaller
training set and fine-tuning only a part of the model, FinBERT outperforms
state-of-the-art machine learning methods.
"""

In [ ]:
question = "What is financial sentiment analysis?"

In [ ]:
qa_bot(question=question, context=context)

{'score': 0.3027523458003998,
 'start': 33,
 'end': 51,
 'answer': 'a challenging task'}

In [ ]:
question = "What is FinBERT?"

In [ ]:
result = qa_bot(question=question, context=context)
print(result["answer"])

a language model 
based on BERT


## Machine Translation

In [ ]:
translater = pipeline("translation_en_to_fr")

No model was supplied, defaulted to google-t5/t5-base and revision a9723ea (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
translater("Hello")

[{'translation_text': 'Bonjour'}]

In [ ]:
translater("Thanks")

[{'translation_text': 'Merci'}]

In [ ]:
sentence = "What is your name?"
translater(sentence)[0]["translation_text"]

'Quel est votre nom?'

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
